In [1]:
import pandas as pd
import numpy as np
import os
files = os.listdir("../data")

In [2]:
def fix_array(x):
    x = np.fromstring(
    x.replace('\n','')
    .replace('[','')
    .replace(']','')
    .replace('  ',' '), sep=' ')
    return x.reshape((1, 768))

In [3]:
qa = pd.read_csv("../data/" + files[0])
for file in files[1:]:
    print(file)
    qa = pd.concat([qa, pd.read_csv("../data/" + file)], axis = 0)
    

healthtapQAs.csv
questionDoctorQAs.csv
webmdQAs.csv
ehealthforumQAs.csv


In [4]:
qa["question_bert"] = qa["question_bert"].apply(fix_array)
qa["answer_bert"] = qa["answer_bert"].apply(fix_array)

In [5]:
qa = qa.sample(frac = 1)
qa.reset_index(inplace = True, drop = True)

In [6]:
question_bert = np.concatenate(qa["question_bert"].values, axis=0)
answer_bert = np.concatenate(qa["answer_bert"].values, axis=0)

In [7]:
answer_bert.shape

(166804, 768)

In [8]:
question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

In [9]:
import faiss
answer_index = faiss.IndexFlatL2(answer_bert.shape[-1])
answer_index.add(answer_bert)

question_index = faiss.IndexFlatL2(question_bert.shape[-1])
question_index.add(question_bert)

In [10]:
k = 4
D1, I1 = answer_index.search(question_bert[0:1].astype('float32'), k)
D2, I2 = question_index.search(question_bert[0:1].astype('float32'), k)
print(I1)
print(D1)
print(I2)
print(D2)

[[160654 164703  30136  17242]]
[[23.75791  27.490116 27.51597  27.605429]]
[[125141      0  55077 142202]]
[[ 0.       0.       0.      17.27046]]


In [11]:
qa["question"][0]

'i have rather white stools the past few days a bit unwell a bit gassy but no pain. i had gastric sleeve surgery 15 months ago. gall bladder?'

In [12]:
qa["answer"][0]

'white stools or stools without their normal color suggests malabsorption of food and can be related to a liver gallbladder or intestinal issues.'

In [13]:
list(I1[0])

[160654, 164703, 30136, 17242]

In [14]:
qa.loc[list(I1[0]), :]

,question,answer,question_bert,answer_bert
160654,i had a nissan fundoplication over 15 years ag...,i had a nissan fundoplication over 15 years ag...,"[[0.0368798859, 0.564586043, -0.127868533, 0.3...","[[0.0565036982, 0.185404703, -0.204453021, 0.1..."
164703,hypothyroidism- can this make your head feel l...,i've been diagnosed recently with hypothyroidi...,"[[-0.415793955, 0.0866782218, 0.0517509356, 0....","[[-0.308385283, 0.173175916, -0.0469668321, 0...."
30136,is it possible hiatal hernia and intercostal m...,my spouse an asthmatic for 45 years recently h...,"[[0.0144768339, 0.320690989, -0.241046146, 0.3...","[[-0.370323479, 0.348526865, -0.448246241, 0.0..."
17242,how long is a cold contagious?,i have had a bad chest cold since friday (toda...,"[[0.0144032426, 0.0714931414, -0.162377745, -0...","[[-0.125337616, 0.288207948, -0.22237289, 0.06..."


In [15]:
qa.loc[list(I2[0]), :]

,question,answer,question_bert,answer_bert
125141,i have rather white stools the past few days a...,indicate lack of pigment (of which bile (made ...,"[[0.0354531258, 0.298021883, -0.208888337, 0.2...","[[0.0608750097, 0.1624486, -0.32438454, 0.1565..."
0,i have rather white stools the past few days a...,white stools or stools without their normal co...,"[[0.0354531258, 0.298021883, -0.208888337, 0.2...","[[-0.265324056, 0.277673572, -0.438004732, -0...."
55077,i have rather white stools the past few days a...,white pale stool may reflect a significant bil...,"[[0.0354531258, 0.298021883, -0.208888337, 0.2...","[[-0.228493288, 0.0831475481, -0.440894783, -0..."
142202,i have had facial flushing for the past 6 mont...,go to see a rheumatologist he or she can sort ...,"[[0.0747706592, 0.258431494, -0.0580760166, 0....","[[0.0863120854, 0.22896187, 0.172008887, 0.161..."
